In [ ]:
from hdf5libs import HDF5RawDataFile

import os
import daqdataformats
import detdataformats
from daqdataformats import FragmentType
from rawdatautils.unpack.daphne import *

import matplotlib.pyplot as plt
import os

In [ ]:
folder    = 'data0'
run       = 'run025130'
nskip     = 1
nrecords  = None #None if you want to run all the folder

det       = 'HD_PDS'

In [ ]:
info_perfragment = []
files = os.listdir(f'/{folder}')

# Define a function to extract fragment information
def extract_fragment_info(frag):
    frag_id = str(frag).split(' ')[3][:-1]
    fragType = frag.get_header().fragment_type
    
    if fragType == FragmentType.kDAPHNE.value:  # For self trigger
        trigger = 'self_trigger'
        timestamps = np_array_timestamp(frag)
        adcs = np_array_adc(frag)
        channels = np_array_channels(frag)
    elif fragType == 13:  # For full_stream
        trigger = 'full_stream'
        timestamps = np_array_timestamp_stream(frag)
        adcs = np_array_adc_stream(frag)
        channels = np_array_channels_stream(frag)[0]
    
    return trigger, frag_id, channels, adcs, timestamps

# Iterate through files
for file in files:
    if file.endswith(('.json', '.json.copied')):
        continue  # Skip JSON files
        
    parts = file.split('_')
    if len(parts) <= 2 or parts[2] != run:
        continue  # Skip files not matching 'run'
    
    filename  = f'/{folder}/{file}'
    h5_file = HDF5RawDataFile(filename)
    records = h5_file.get_all_record_ids()
    
    if nrecords is None: nrecord = len(records)
    # Iterate through records
    for r in records[nskip:nrecords]:
        pds_geo_ids = list(h5_file.get_geo_ids_for_subdetector(r, detdataformats.DetID.string_to_subdetector(det)))
        
        # Iterate through geo_ids
        for gid in pds_geo_ids:
            frag = h5_file.get_frag(r, gid)
            trigger, frag_id, channels, adcs, timestamps = extract_fragment_info(frag)
            info_perfragment.append([trigger, frag_id, gid, channels, adcs, timestamps])

In [ ]:
mode = 'self_trigger' #'self_trigger' 'full_stream

In [ ]:
### Waveforms per fragment & timestamps

# Get unique geo_ids
gid = list(set(item[1] for item in info_perfragment))

for index, item in enumerate(info_perfragment):
    trigger, fragment, geo_id, channels, adcs, timestamps = item

    if mode == trigger:
        if trigger == 'self_trigger':
            for i, wf in enumerate(adcs):
                plt.plot(wf, label=channels[i])

        else:
            ch_ref = list(set(channels))
            waveforms = [[] for _ in range(len(ch_ref))]

            for adc_vector in adcs:
                for adc_index, wf_element in enumerate(adc_vector):
                    ch_index = ch_ref.index(channels[adc_index])
                    waveforms[ch_index].append(int(wf_element))

            for i, wf in enumerate(waveforms):
                plt.plot(wf, label=ch_ref[i])
            
        plt.title(f'Fragment: {fragment}')
        plt.legend()
        plt.show()

In [ ]:
#### Persistence of each channel per geo_id
from scipy.signal import savgol_filter
import numpy as np

gid = list(set([item[2] for item in info_perfragment if item[0] == mode]))

for g in gid:
    wf_list = [[] for _ in range(50)]
    for index, item in enumerate(info_perfragment):
        trigger, fragment, geo_id, channels, adcs, timestamps = item
        
        if trigger == mode and geo_id ==g:    
            
            if mode == 'full_stream':
                list_channels = list(set(channels))
                waveforms = [[] for _ in range(len(list_channels))]
    
                for adc_vector in adcs:
                    for adc_index, wf_element in enumerate(adc_vector):
                        ch_index = list_channels.index(channels[adc_index])
                        waveforms[ch_index].append(int(wf_element))

            if mode == 'self_trigger':
                waveforms = adcs

            for i, wf in enumerate(waveforms):
                ch_index = channels[i]
                if len(wf_list[ch_index]) < 200:
                    wf_list[ch_index].append(wf)
            
    if wf_list is not None:
        for wf_index, wf in enumerate(wf_list):
                    if len(wf) > 0:
                        fig, ax = plt.subplots(1, 2, figsize=(10,4))
                        for w in wf:
                            ax[0].plot(w)
                            ax[0].set_title(f'ID: {g} Channel = {wf_index}')
                            w_filtered = savgol_filter(w, 5, 1)
                            ax[1].plot(w_filtered - w_filtered[:100].mean(), alpha = 0.4)
                            ax[1].set_ylim(-20, 20)
                            ax[1].set_xlim(0, 500)
                        plt.title(f'ID: {g} Channel = {wf_index}')
                        plt.show()